## TV Commercial Detection
Below are the steps to import the source data for pre-processing before modelling in MATLAB.

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import string
import re
# https://pypi.org/project/libsvm/
!pip install libsvm
from libsvm.svmutil import *

     |████████████████████████████████| 174kB 1.9MB/s eta 0:00:01
  Created wheel for libsvm: filename=libsvm-3.23.0.3-cp37-cp37m-macosx_10_9_x86_64.whl size=186565 sha256=ccb45589735a7d38c48f76ba2418dc45e1a54c4db0a6304cd4f3c8bb07cdc98b
  Stored in directory: /Users/totoro/Library/Caches/pip/wheels/59/46/96/b59105dea82f698d348bf0b5ffbbe2dfa1f2c8a6e44c9ed69c
Successfully built libsvm


In [2]:
import pickle
# Function to serialise object
def save_pickle(file_nm, obj):
    pickle_out = open(file_nm, 'wb')
    pickle.dump(obj, pickle_out)
    pickle_out.close()

# Function to load object from pickle file
def load_pickle(file_nm):
    pickle_in = open(file_nm, 'rb')
    obj = pickle.load(pickle_in)
    return obj

### Read in the source file and wrangle into a data frame

In [3]:
src_y, src_x = svm_read_problem('BBC.txt')

In [14]:
src_y_df = pd.DataFrame(src_y)
src_x_df = pd.DataFrame(src_x)
src_df = pd.concat([src_y_df, src_x_df], axis=1)
src_df.tail(10)

,0,1,2,3,4,5,6,7,8,9,...,779,128,689,1119,324,694,88,785,198,487
17710,1.0,25.0,2.261500,1.149089,7.639838,10.709817,0.008558,0.006488,0.074750,0.050990,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17711,1.0,38.0,1.178209,0.543506,5.314273,1.886085,0.009857,0.010259,0.088651,0.048706,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17712,1.0,28.0,0.963389,1.327245,18.657602,27.602407,0.015765,0.009704,0.104464,0.083542,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17713,1.0,25.0,2.333455,1.937060,13.175022,5.996551,0.012585,0.007584,0.096125,0.055886,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17714,1.0,25.0,3.525863,1.676093,14.641719,6.890201,0.012671,0.008254,0.099250,0.050228,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17715,1.0,66.0,2.685833,3.018033,13.668181,17.187395,0.014353,0.007819,0.119271,0.060057,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17716,1.0,33.0,1.597726,1.008243,5.417785,2.954453,0.015893,0.007553,0.125758,0.055514,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17717,1.0,25.0,1.465399,0.702240,5.848524,3.574110,0.019303,0.009761,0.100125,0.058169,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17718,1.0,88.0,0.668713,1.235713,4.243607,10.771485,0.018320,0.006595,0.111009,0.043714,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17719,1.0,47.0,0.218305,0.197495,1.000669,0.848586,0.019544,0.009338,0.105120,0.057398,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
#Set column names
col_ls = ['x' + str(col) for col in src_df.columns]
col_ls[0] = 'y'
src_df.columns = col_ls
src_df.to_excel('src_df.xlsx', index=False)

In [11]:
src_x_df = pd.DataFrame(src_x)
src_x_df.tail(20)

,1,2,3,4,5,6,7,8,9,10,...,779,128,689,1119,324,694,88,785,198,487
17700,32.0,5.213313,3.320301,22.508598,9.201363,0.013277,0.008355,0.122656,0.089230,3485.840820,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17701,37.0,3.350123,2.014289,18.315825,10.813235,0.013457,0.008184,0.135304,0.085303,3549.521973,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17702,39.0,3.598186,2.152019,16.870825,7.124610,0.009747,0.005761,0.085497,0.055139,3521.955566,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17703,45.0,1.083410,0.470376,3.768514,1.293194,0.004589,0.004470,0.073681,0.037598,3492.497803,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17704,32.0,0.668095,2.325667,6.086069,18.265038,0.011821,0.013577,0.071680,0.041501,2887.685303,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17705,29.0,5.100513,2.286654,24.867905,10.272936,0.014824,0.011046,0.100323,0.051987,3442.322021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17706,28.0,1.091611,0.624903,4.027010,1.907658,0.007669,0.005806,0.106250,0.062611,3391.953613,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17707,35.0,0.480898,0.225287,3.318975,1.568225,0.009752,0.007462,0.076607,0.049985,3492.553955,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17708,71.0,3.115963,1.146667,7.308737,2.572871,0.012628,0.009362,0.108935,0.052561,3748.644775,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17709,25.0,1.608348,0.834651,5.811115,4.868095,0.006897,0.005793,0.094125,0.058902,3319.768066,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
# Read the source file (text file) with values arranged in SVM Lib format
src_data_txt_ls = []
src_file_path = 'BBC.txt'
with open(src_file_path) as fp:
    for i, line in enumerate(fp):
        # Read from line 10 after header
        if i >= 10:
            src_data_txt_ls += [line]
            
print(src_data_txt_ls[-1])

# Split each value by delimiter ' '
src_data_txt_ls = [line.split(' ') for line in src_data_txt_ls]
# Strip end of line break \n
src_data_txt_ls = [[val.replace('\n', '') for val in line] for line in src_data_txt_ls]
# Strip blank list elements
src_data_txt_ls = [[val.replace('\n', '') for val in line] for line in src_data_txt_ls if len(line) > 0]
# Convert each value from text in source data
def get_dct(s):
    delimiter_pos = s.find(':')
    # If ':' exists then convert the string into dictionary
    if delimiter_pos > 0:
        # Extract key value before ':' delimiter
        key = s[:delimiter_pos]
        # Extract the floating point value after ':' delimiter
        val = float(s[delimiter_pos+1:])
        dct = {key: [val]}
    # Create a key for the target variable
    elif delimiter_pos <= 0:
        # Convert classification '-1' or '1' to numeric, or 0 where it is blank
        if len(s) >= 0:
            dct = {'0': [s]}
        else:
            dct = {'0': ['0']}
    return dct

src_data_txt_ls = [[get_dct(val) for val in row] for row in src_data_txt_ls]

print(src_data_txt_ls[-1])

1  1:47 2:0.218305 3:0.197495 4:1.000669 5:0.848586 6:0.019544 7:0.009338 8:0.105120 9:0.057398 10:3648.663574 11:1113.372803 12:7117.023438 13:2172.832520 14:1397.162354 15:1476.380493 16:115.411110 17:55.871628 18:0.829607 19:0.053511 20:0.014346 21:0.005305 22:0.002199 23:0.001299 24:0.001081 25:0.000852 26:0.000651 27:0.000737 28:0.000616 29:0.000639 30:0.000652 31:0.000642 32:0.000463 33:0.000320 34:0.000303 35:0.000368 36:0.000383 37:0.000252 38:0.000170 39:0.000099 40:0.000042 41:0.000030 42:0.000024 43:0.000028 44:0.000029 45:0.000019 46:0.000028 47:0.000020 48:0.000014 49:0.000011 50:0.000011 51:0.000011 52:0.000015 53:0.000020 54:0.000022 55:0.000042 56:0.000028 57:0.000001 58:0.897549 59:0.012759 60:0.001295 61:0.000326 62:0.000146 63:0.000103 64:0.000100 65:0.000093 66:0.000105 67:0.000097 68:0.000100 69:0.000094 70:0.000104 71:0.000102 72:0.000082 73:0.000063 74:0.000059 75:0.000080 76:0.000080 77:0.000087 78:0.000091 79:0.000093 80:0.000115 81:0.000156 82:0.000259 83:0.00

In [26]:
# Get rid of duplicate '0' dictrionary key parsed
for dct_ls in src_data_txt_ls:
    # If second entries of key '0' is blank, remove the item
    if (dct_ls[1].get('0') == ['']):
        dct_ls.pop(1)

print(src_data_txt_ls[-1])

[{'0': ['1']}, {'1': [47.0]}, {'2': [0.218305]}, {'3': [0.197495]}, {'4': [1.000669]}, {'5': [0.848586]}, {'6': [0.019544]}, {'7': [0.009338]}, {'8': [0.10512]}, {'9': [0.057398]}, {'10': [3648.663574]}, {'11': [1113.372803]}, {'12': [7117.023438]}, {'13': [2172.83252]}, {'14': [1397.162354]}, {'15': [1476.380493]}, {'16': [115.41111]}, {'17': [55.871628]}, {'18': [0.829607]}, {'19': [0.053511]}, {'20': [0.014346]}, {'21': [0.005305]}, {'22': [0.002199]}, {'23': [0.001299]}, {'24': [0.001081]}, {'25': [0.000852]}, {'26': [0.000651]}, {'27': [0.000737]}, {'28': [0.000616]}, {'29': [0.000639]}, {'30': [0.000652]}, {'31': [0.000642]}, {'32': [0.000463]}, {'33': [0.00032]}, {'34': [0.000303]}, {'35': [0.000368]}, {'36': [0.000383]}, {'37': [0.000252]}, {'38': [0.00017]}, {'39': [9.9e-05]}, {'40': [4.2e-05]}, {'41': [3e-05]}, {'42': [2.4e-05]}, {'43': [2.8e-05]}, {'44': [2.9e-05]}, {'45': [1.9e-05]}, {'46': [2.8e-05]}, {'47': [2e-05]}, {'48': [1.4e-05]}, {'49': [1.1e-05]}, {'50': [1.1e-05]}

In [27]:
# For each row, combine all dictionaries into one, then create a data frame
src_df_ls = [pd.DataFrame(dict(pair for d in ls for pair in d.items())) for ls in src_data_txt_ls]
src_df_ls[0].head()

,0,1,2,3,4,5,6,7,8,9,...,559,573,621,627,767,840,877,1016,4124,4125
0,1,35.0,5.54137,3.746644,17.127348,8.905652,0.013702,0.012665,0.16125,0.061079,...,0.116667,0.091667,0.016667,0.05,0.183333,0.008333,0.241667,0.058333,0.254915,0.615579


In [28]:
src_df_ls[-1].head()

,0,1,2,3,4,5,6,7,8,9,...,572,573,580,623,709,762,816,1002,4124,4125
0,1,47.0,0.218305,0.197495,1.000669,0.848586,0.019544,0.009338,0.10512,0.057398,...,0.113095,0.285714,0.136905,0.184524,0.011905,0.065476,0.017857,0.005952,0.174725,0.221808


In [29]:
# Get all possible column values
all_cols_ls = []
for df in src_df_ls:
    # Extract column names not already in the all data frames list
    cols_not_in_ls = [col for col in df.columns if col not in all_cols_ls]
    if len(cols_not_in_ls) > 0:
        # Append to the all data frame list
        all_cols_ls += cols_not_in_ls

# Remove duplicate column names
all_cols_ls = list(set(all_cols_ls))

# Convert column names to integer and then sort, before changing back to string
all_cols_int_ls = [int(nm) for nm in all_cols_ls]
all_cols_int_ls.sort()
all_cols_ls = [str(col) for col in all_cols_int_ls]

# Preview
all_cols_ls[:5]

['0', '1', '2', '3', '4']

In [30]:
# Create the complete data frame for one row
src_df = pd.DataFrame([np.nan] * len(all_cols_ls), index=all_cols_ls).T

# Copy the rows by the number of data frames to combine
src_df = pd.concat([src_df] * len(src_df_ls))

# Reset index
src_df.reset_index(drop=True, inplace=True)

print(src_df.shape)

src_df.head()

(17710, 231)


,0,1,2,3,4,5,6,7,8,9,...,924,959,1002,1016,1028,1048,1112,1119,4124,4125
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
src_df.head(20)

,0,1,2,3,4,5,6,7,8,9,...,924,959,1002,1016,1028,1048,1112,1119,4124,4125
0,1,35.0,5.541370,3.746644,17.127348,8.905652,0.013702,0.012665,0.161250,0.061079,...,NaN,NaN,NaN,0.058333,NaN,NaN,NaN,NaN,0.254915,0.615579
1,1,56.0,4.980148,2.165492,18.260698,9.478476,0.013703,0.006267,0.115402,0.125917,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.893830,0.576667
2,1,44.0,0.985358,1.133078,5.623209,7.615088,0.013907,0.005289,0.067543,0.026143,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.441795,0.632923
3,1,46.0,5.098480,4.299145,16.210764,9.947581,0.012770,0.009053,0.110122,0.062959,...,NaN,0.006098,0.036585,0.018293,NaN,0.067073,NaN,NaN,0.516036,0.464025
4,1,30.0,4.476842,1.987119,12.356640,7.803830,0.011803,0.008397,0.094479,0.060629,...,NaN,0.030000,NaN,NaN,NaN,0.010000,NaN,NaN,0.778929,0.081424
5,1,41.0,4.944958,2.554706,19.797184,7.313418,0.014482,0.009869,0.093445,0.055327,...,0.020833,0.006944,0.048611,0.006944,NaN,0.034722,NaN,NaN,0.019725,0.061096
6,1,45.0,5.340932,2.193098,17.277145,5.847475,0.014733,0.008707,0.111319,0.088363,...,0.018750,0.012500,NaN,0.031250,NaN,0.031250,NaN,NaN,0.512529,0.307446
7,1,45.0,5.802182,2.908596,14.641971,6.270773,0.019044,0.010232,0.091181,0.056687,...,0.006250,NaN,0.018750,0.025000,NaN,0.006250,NaN,NaN,0.066710,0.315586
8,1,25.0,0.834001,1.674893,5.456865,8.578226,0.015192,0.009498,0.132875,0.090328,...,NaN,NaN,NaN,0.125000,NaN,0.025000,NaN,NaN,0.078167,0.816183
9,1,57.0,3.486077,1.928280,11.625623,6.826046,0.015305,0.010998,0.145504,0.091697,...,0.004808,NaN,NaN,0.028846,NaN,0.076923,NaN,NaN,0.045920,0.846529


 Columns per specifications:
 https://archive.ics.uci.edu/ml/datasets/TV+News+Channel+Commercial+Detection+Dataset
 
 Labels : - +1/-1 ( Commercials/Non Commercials) 
 Feature 
 Dimension Index in feature File 
 Shot Length 
 1 
 Motion Distribution( Mean and Variance) 
 2 - 3 
 Frame Difference Distribution ( Mean and Variance) 
 4 - 5 
 Short time energy ( Mean and Variance) 
 6 - 7 
 ZCR( Mean and Variance) 
 8 - 9 
 Spectral Centroid ( Mean and Variance) 
 10 - 11 
 Spectral Roll off ( Mean and Variance) 
 12 - 13 
 Spectral Flux ( Mean and Variance) 
 14 - 15 
 Fundamental Frequency ( Mean and Variance) 
 16 - 17 
 Motion Distribution ( 40 bins) 
 18 - 58 
 Frame Difference Distribution ( 32 bins) 
 59 - 91 
 Text area distribution ( 15 bins Mean and 15 bins for variance ) 
 92 - 122 
 Bag of Audio Words ( 4000 bins) 
 123 - 4123 
 Edge change Ratio ( Mean and Variance) 
 4124 - 4125 

In [33]:
# Save the data frame object as pickle file to avoid re-loading each time
save_pickle('src_df.pickle', src_df)

In [34]:
src_desc_df = src_df.describe(include='all').T

In [35]:
src_df.head()

,0,1,2,3,4,5,6,7,8,9,...,924,959,1002,1016,1028,1048,1112,1119,4124,4125
0,1,35.0,5.541370,3.746644,17.127348,8.905652,0.013702,0.012665,0.161250,0.061079,...,NaN,NaN,NaN,0.058333,NaN,NaN,NaN,NaN,0.254915,0.615579
1,1,56.0,4.980148,2.165492,18.260698,9.478476,0.013703,0.006267,0.115402,0.125917,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.893830,0.576667
2,1,44.0,0.985358,1.133078,5.623209,7.615088,0.013907,0.005289,0.067543,0.026143,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.441795,0.632923
3,1,46.0,5.098480,4.299145,16.210764,9.947581,0.012770,0.009053,0.110122,0.062959,...,NaN,0.006098,0.036585,0.018293,NaN,0.067073,NaN,NaN,0.516036,0.464025
4,1,30.0,4.476842,1.987119,12.356640,7.803830,0.011803,0.008397,0.094479,0.060629,...,NaN,0.030000,NaN,NaN,NaN,0.010000,NaN,NaN,0.778929,0.081424


It would appear that the target variable is relatively balanced.

In [36]:
# Check dimension of data frame
print(src_df.shape)

# Export file to Excel
src_df.to_csv('src_df.csv', index=False)

(17710, 231)
